In [67]:
import pandas as pd
import codecs
import json
from collections import defaultdict

def load_data(file_path):
  """
  Loads data from a CSV or JSON file and returns a dictionary
  where keys are (doc_id, par_id, sent_id) tuples and values are
  lists of terms.

  Args:
    file_path: The path to the input file (CSV or JSON).

  Returns:
    A dictionary containing the loaded data.

  Raises:
    ValueError: If the file format is not supported.
  """
  if file_path.endswith('.csv'):
    # Load data from CSV file
    df = pd.read_csv(file_path)
    df.fillna('', inplace=True) # Fill NaN values with empty strings
    data = defaultdict(list)
    # Iterate over rows and extract terms
    for doc_id, par_id, sent_id, _, term in df.itertuples(index=False):
      if term.strip() != '':
        data[(doc_id, par_id, sent_id)].append(term.strip())
      else:
         data[(doc_id, par_id, sent_id)]
  elif file_path.endswith('.json'):
    # Load data from JSON file
    with codecs.open(file_path, 'r', 'utf-8') as f:
      json_data = json.load(f)
    # Extract terms from JSON data
    data = {(row["document_id"], row["paragraph_id"], row["sentence_id"]): row["term_list"]
            for row in json_data["data"]}
  else:
    # Raise error for unsupported file formats
    raise ValueError("Unsupported file format. Only CSV and JSON files are supported.")
  return data

In [68]:
def micro_f1_score(gold_standard, system_output):
  """
  Evaluates a term extraction system's performance using Precision, Recall,
  and F1 score based on individual term matching (micro-average).

  Args:
    gold_standard: A list of lists, where each inner list contains the
                   gold standard terms for an item.
    system_output: A list of lists, where each inner list contains the
                   terms extracted by the system for the corresponding item.

  Returns:
    A tuple containing the Precision, Recall, and F1 score.
  """
  total_true_positives = 0
  total_false_positives = 0
  total_false_negatives = 0

  # Iterate through each item's gold standard and system output terms
  for gold, system in zip(gold_standard, system_output):
    # Convert to sets for efficient comparison
    gold_set = set(gold)
    system_set = set(system)

    # Calculate True Positives, False Positives, and False Negatives for the current item
    true_positives = len(gold_set.intersection(system_set))
    false_positives = len(system_set - gold_set)
    false_negatives = len(gold_set - system_set)

    # Accumulate totals across all items
    total_true_positives += true_positives
    total_false_positives += false_positives
    total_false_negatives += false_negatives

  # Calculate Precision, Recall, and F1 score (micro-average)
  precision = total_true_positives / (total_true_positives + total_false_positives) if (total_true_positives + total_false_positives) > 0 else 0
  recall = total_true_positives / (total_true_positives + total_false_negatives) if (total_true_positives + total_false_negatives) > 0 else 0
  f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

  return precision, recall, f1

In [69]:
def type_f1_score(gold_standard, system_output):
  """
  Evaluates a term extraction system's performance using Type Precision,
  Type Recall, and Type F1 score based on the set of unique terms extracted
  at least once across the entire dataset.

  Args:
    gold_standard: A list of lists, where each inner list contains the
                   gold standard terms for an item.
    system_output: A list of lists, where each inner list contains the
                   terms extracted by the system for the corresponding item.

  Returns:
    A tuple containing the Type Precision, Type Recall, and Type F1 score.
  """

  # Get the set of all unique gold standard terms across the dataset
  all_gold_terms = set()
  for item_terms in gold_standard:
    all_gold_terms.update(item_terms)

  # Get the set of all unique system extracted terms across the dataset
  all_system_terms = set()
  for item_terms in system_output:
    all_system_terms.update(item_terms)

  # Calculate True Positives (terms present in both sets)
  type_true_positives = len(all_gold_terms.intersection(all_system_terms))

  # Calculate False Positives (terms in system output but not in gold standard)
  type_false_positives = len(all_system_terms - all_gold_terms)

  # Calculate False Negatives (terms in gold standard but not in system output)
  type_false_negatives = len(all_gold_terms - all_system_terms)

  # Calculate Type Precision, Type Recall, and Type F1 score
  type_precision = type_true_positives / (type_true_positives + type_false_positives) if (type_true_positives + type_false_positives) > 0 else 0
  type_recall = type_true_positives / (type_true_positives + type_false_negatives) if (type_true_positives + type_false_negatives) > 0 else 0
  type_f1 = 2 * (type_precision * type_recall) / (type_precision + type_recall) if (type_precision + type_recall) > 0 else 0

  return type_precision, type_recall, type_f1

In [70]:
GOLD_STANDARD_PATH = "../data/subtask_a_dev.json"

In [71]:
# Load data from the specified file paths
gold_standard_dict = load_data(GOLD_STANDARD_PATH)

# Extract the term lists from the loaded data dictionaries
gold_standard = list(gold_standard_dict.values())

Dataset	
Micro-Precision	Micro-Recall	Micro-F1	Type-Precision	Type-Recall	  Type-F1
	0.439	      0.616	         0.513	       0.372	       0.636	    0.47


In [72]:
import os
import glob

# Path to predictions directory and baseline
predictions_dir = "../src/predictions/"
gold_standard_file = "../data/subtask_a_dev.json"

# Get all JSON files in the predictions directory
prediction_files = glob.glob(os.path.join(predictions_dir, "*.json"))


# Load gold standard once
gold_standard_dict = load_data(gold_standard_file)
gold_standard = list(gold_standard_dict.values())

# Store results
results = []

# Evaluate each prediction file
for pred_file in prediction_files:
    if pred_file.find("subtask_a_dev_") == -1:
        continue
    file_name = os.path.basename(pred_file)
    #print(f"Evaluating {file_name}...")
    
    try:
        # Load system output
        system_output_dict = load_data(pred_file)
        system_output = list(system_output_dict.values())
        
        # Calculate micro-averaged metrics
        precision, recall, f1 = micro_f1_score(gold_standard, system_output)
        
        # Calculate type metrics
        type_precision, type_recall, type_f1 = type_f1_score(gold_standard, system_output)
        
        # Store results with cleaned file name
        clean_name = file_name.replace("subtask_a_dev_", "").replace("_preds", "").replace(".json", "")
        results.append({
            'file_name': clean_name,
            'micro_precision': round(precision, 3),
            'micro_recall': round(recall, 3),
            'micro_f1': round(f1, 3),
            'type_precision': round(type_precision, 3),
            'type_recall': round(type_recall, 3),
            'type_f1': round(type_f1, 3)
        })
    except Exception as e:
        print(f"Error evaluating {file_name}: {e}")

# Create DataFrame and sort by micro F1 score (descending)
results_df = pd.DataFrame(results)
results_df = results_df.sort_values('micro_f1', ascending=False).reset_index(drop=True)

# Display the table
print("\n" + "="*100)
print("EVALUATION RESULTS")
print("="*100)
print(results_df.to_string(index=False))

# Save to CSV
output_csv = "subtask_a_evaluation_results.csv"
results_df.to_csv(output_csv, index=False)
print(f"\nResults saved to {output_csv}")


EVALUATION RESULTS
                                          file_name  micro_precision  micro_recall  micro_f1  type_precision  type_recall  type_f1
 ensemble_bert_2e-5_change_spacy_dictfilter_refined            0.770         0.734     0.751           0.728        0.674    0.700
        ensemble_bert_2e-5_changed_spacy_dictfilter            0.769         0.729     0.749           0.728        0.674    0.700
            ensemble_bert_2e-5_changed_reranked_llm            0.769         0.723     0.745           0.730        0.669    0.698
ensemble_bert_2e-5_change_spacy_baseline_dictfilter            0.755         0.736     0.745           0.703        0.686    0.695
    ensemble_bert_2e-5_changed_reranked_llm_refined            0.782         0.701     0.739           0.760        0.653    0.702
              ensemble_bert_2e-5_cased_reranked_llm            0.756         0.714     0.734           0.695        0.669    0.682
          ensemble_bert_2e-5_cased_spacy_dictfilter            